# code for generating and saving invoices
along with corresponding target arrays and truth values


This is the second version that still holds "nested csv's as serialized strings", but now we know how to eval() those into the original objects again later....

In [61]:
# rmember to git pull the latest templates from 
# https://github.com/riklmr/MLiFC_invoice_classifier
# if you have not already (where else did this notebook come from?)

In [62]:
!ls

Baseline Ortec.ipynb			   README.md
invoice_from_disk_sample_code.ipynb	   RegExDragon.ipynb
invoices_train.zipNOT			   templates
Jannes_Baseline_Invoices_ToFromDisk.ipynb  train
Jannes_Baseline_Ortec.ipynb		   Wk6_Ortec_IBAN_generation.ipynb
LICENSE


In [51]:
# this trick allows you to change underlying libraries on the fly

%reload_ext autoreload
%autoreload 2   # The 2 is important Rik!


In [52]:
import templates.invoicegen

import numpy as np
import pandas as pd
import glob
# import uuid


In [53]:
'''
Ignore:           0
Sender Name:      1 
Sender KVK:       2 
Sender IBAN:      3 
Invoice Reference:4
Total:            5
'''

train_dir = "train/"
! mkdir -p {train_dir}


In [55]:
# Create plenty of invoices for each template
templates_all = glob.glob("./templates/TEMPLATE_*.txt")
num_templates = len(templates_all)
PLENTY = 100000 // num_templates
print("Creating {} invoices times {} templates = {} in total".format(PLENTY, num_templates, PLENTY * num_templates))
print("Saving in directory {} ...".format(train_dir))

for txtname in templates_all:
    print("from template {}".format(txtname))
    with open(txtname, "r") as content_file:
        content = content_file.read()
        # Create invoices from template
        for i in range(PLENTY):
            # create_content() returns [invoice, target, truth]
            # invoice is string
            # target is array of classlabels per char in invoice
            # truth is array [uuid, SenderName, SenderKVK, SenderIBAN, Reference, Total]
            invoice, target, truth = templates.invoicegen.create_invoice(content)

            mysample = pd.DataFrame(columns = ['invoice', 'target', 'truth'])
            mysample.loc[0, 'invoice'] = invoice       # invoice already is a string
            mysample.loc[0, 'target']  = str(target)   # serialize array here
            mysample.loc[0, 'truth']   = str(truth)    # serialize array here

            uuid = truth[0]
            fname = train_dir + uuid + '.csv'
            # print("saving {} ...".format(fname))
            mysample.to_csv(fname, index=False)
        #
    #
#
print{"done"}


Creating 4761 invoices times 21 templates = 99981 in total
Saving in directory train/ ...
from template ./templates/TEMPLATE_1.txt
from template ./templates/TEMPLATE_10.txt
from template ./templates/TEMPLATE_8.txt
from template ./templates/TEMPLATE_2.txt
from template ./templates/TEMPLATE_9.txt
from template ./templates/TEMPLATE_20.txt
from template ./templates/TEMPLATE_4.txt
from template ./templates/TEMPLATE_18.txt
from template ./templates/TEMPLATE_17.txt
from template ./templates/TEMPLATE_11.txt
from template ./templates/TEMPLATE_5.txt
from template ./templates/TEMPLATE_7.txt
from template ./templates/TEMPLATE_14.txt
from template ./templates/TEMPLATE_13.txt
from template ./templates/TEMPLATE_19.txt
from template ./templates/TEMPLATE_12.txt
from template ./templates/TEMPLATE_3.txt
from template ./templates/TEMPLATE_16.txt
from template ./templates/TEMPLATE_21.txt
from template ./templates/TEMPLATE_6.txt
from template ./templates/TEMPLATE_15.txt


In [56]:
# ! rm -r {train_dir}
! du -sh {train_dir}

726M	train/


In [ ]:
! zip -qr invoices_train.zipNOT ./train

In [60]:
! ls -l invoices_train*

-rw-rw-r-- 1 paperspace paperspace 118096904 Mar 16 09:47 invoices_train.zipNOT
